## 用numpy写一个神经网络实现MINST

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets
from torchvision.transforms import ToTensor

利用pytorch下载的数据集，稍后会转换成ndarray给numpy使用

In [ ]:
# 指定函数的返回类型，方便IDE进行提示
def get_data() -> tuple[np.ndarray, np.ndarray]:
    training_data = datasets.MNIST(
        root="data",
        train=True,
        download=True,
        transform=ToTensor()
    )
    test_data = datasets.FashionMNIST(
        root="data",
        train=False,
        download=True,
        transform=ToTensor()
    )
    tr = training_data.data / 255
    te = training_data.targets
    return tr.numpy(), te.numpy()

初始化网络，输入是应该是28*28=784 
第一层输出50， 第二层输出100
最后一层即第三层输出10个数，代表结果，输出函数可以用softmax

In [ ]:
def init_network(size_x):
    nn = {
        'w1': np.random.randn(size_x, 50),
        #这里用rand(50)也行，加法的广播机制
        'b1': np.random.randn(1, 50), 

        'w2': np.random.randn(50, 100),
        'b2': np.random.randn(1, 100),

        'w3': np.random.randn(100, 10),
        'b3': np.random.randn(1, 10),
    }
    return nn


激活函数，输出函数

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmax(x):
    c = np.max(x)
    exp_a = np.exp(x - c) # 减去c是为了防止溢出
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y

正向运行网络，推理

In [ ]:
def forward(nn, x):
    w1, w2, w3 = nn['w1'], nn['w2'], nn['w3'] 
    b1, b2, b3 = nn['b1'], nn['b2'], nn['b3']
    
    a1 = x.dot(w1) + b1
    y1 = sigmoid(a1)
    
    a2 = y1.dot(w2) + b2
    y2 = sigmoid(a2)
    
    a3 = y2.dot(w3) + b3
    y3 = softmax(a3)
    
    return y3


运行一次网络

In [ ]:
data, labels = get_data()
data = data.reshape(60000, 28 * 28)
i = np.random.randint(60000)
x = data[i]
t_index = labels[i]
nn = init_network(x.size)
y = forward(nn, x)
y

接下来，定义两种损失函数

In [ ]:
# 均方误差 输出与目标的差的平方和，除以2
def mean_square_error(y, t):
    return 0.5 * np.sum((y - t) ** 2)

# 交叉熵误差: 目标*loge(y)的和的负数
def cross_entropy_error(y, t):
    # 由于loge(0)是负无穷大-inf，计算机无法继续之后的运算
    # 所以给输入增加一个微小的数，并且不影响结果
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))


看一下两种损失函数

In [ ]:
t = np.zeros(10)
t[int(t_index)] = 1

e1 = mean_square_error(y, t)
e2 = cross_entropy_error(y, t)

# 设置np的打印选项：精度最大为小数点后2位，不用科学计数法
np.set_printoptions(precision=2, suppress=True)

print('t =', t)
print('y =', y)
print(f'e1 = {e1:5f}')
print(f'e2 = {e2:5f}')
